In [ ]:
# pip install scapy==2.5.0

  DEPRECATION: Building 'scapy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'scapy'. Discussion can be found at https://github.com/pypa/pip/issues/6334



     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     -------- ------------------------------- 0.3/1.3 MB ? eta -:--:--
     ---------------- ----------------------- 0.5/1.3 MB 1.2 MB/s eta 0:00:01
     ------------------------ --------------- 0.8/1.3 MB 1.2 MB/s eta 0:00:01
     -------------------------------- ------- 1.0/1.3 MB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444458 sha256=ea13502489a8077f3e05d5df61061c6b8c0071f6c97b75de8b9eec8a1fd5d2de
  Stored in directory: c:\users\kimo store\appdata\local\pip\cache\wheels\82\b7\03\8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully 

In [11]:
from scapy.all import sniff, TCP, UDP, IP
from collections import defaultdict
from datetime import datetime
import time
import csv
import joblib
import socket
import numpy as np
from tensorflow.keras.models import load_model


In [12]:
# Load model and transformers
model = load_model("dnn_ids_model.h5")
scaler = joblib.load("scaler.pkl")
le_protocol = joblib.load("le_protocol.pkl")
le_service = joblib.load("le_service.pkl")
le_flag = joblib.load("le_flag.pkl")


In [13]:
OUTPUT_FILE = "features_prediction_log.csv"
FLOW_TIMEOUT = 10
flows = defaultdict(list)
flow_start_time = {}

In [14]:
# Header based on extractable features + prediction
HEADERS = [
    'timestamp', 'src', 'dst', 'sport', 'dport',
    'protocol_type', 'service', 'flag',
    'duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'prediction'
]

In [15]:
def get_service(port):
    try:
        return socket.getservbyport(port)
    except:
        return 'other'

In [16]:
def extract_flow_features(flow_key, pkts):
    src, dst, sport, dport, proto = flow_key
    start_time = pkts[0].time
    end_time = pkts[-1].time
    duration = end_time - start_time

    src_bytes = sum(len(p) for p in pkts if IP in p and p[IP].src == src)
    dst_bytes = sum(len(p) for p in pkts if IP in p and p[IP].src == dst)

    service = get_service(dport)
    land = int(src == dst and sport == dport)
    urgent = sum(1 for p in pkts if TCP in p and p[TCP].flags & 0x20)
    wrong_fragment = sum(1 for p in pkts if IP in p and (p[IP].frag > 0 or p[IP].flags & 0x1))

    flag = 'OTH'
    for p in pkts:
        if TCP in p:
            flag = str(p[TCP].flags)
            break

    # Partial logic for statistical features (simplified)
    count = len(pkts)
    serror_rate = sum(1 for p in pkts if TCP in p and 'S' in str(p[TCP].flags) and 'A' not in str(p[TCP].flags)) / count
    rerror_rate = sum(1 for p in pkts if TCP in p and 'R' in str(p[TCP].flags)) / count

    features = {
        'timestamp': datetime.fromtimestamp(end_time).isoformat(),
        'src': src, 'dst': dst, 'sport': sport, 'dport': dport,
        'protocol_type': proto, 'service': service, 'flag': flag,
        'duration': duration, 'src_bytes': src_bytes, 'dst_bytes': dst_bytes,
        'land': land, 'wrong_fragment': wrong_fragment, 'urgent': urgent,
        'count': count, 'srv_count': count,
        'serror_rate': serror_rate, 'srv_serror_rate': serror_rate,
        'rerror_rate': rerror_rate, 'srv_rerror_rate': rerror_rate,
        'same_srv_rate': 1.0, 'diff_srv_rate': 0.0,
        'srv_diff_host_rate': 0.0, 'dst_host_count': 1,
        'dst_host_srv_count': 1, 'dst_host_same_srv_rate': 1.0,
        'dst_host_diff_srv_rate': 0.0, 'dst_host_same_src_port_rate': 1.0,
        'dst_host_srv_diff_host_rate': 0.0,
        'dst_host_serror_rate': serror_rate,
        'dst_host_srv_serror_rate': serror_rate,
        'dst_host_rerror_rate': rerror_rate,
        'dst_host_srv_rerror_rate': rerror_rate,
    }

    # Predict
    feature_vector = [
        le_protocol.transform([proto])[0] if proto in le_protocol.classes_ else -1,
        le_service.transform([service])[0] if service in le_service.classes_ else -1,
        le_flag.transform([flag])[0] if flag in le_flag.classes_ else -1,
        duration, src_bytes, dst_bytes, land, wrong_fragment, urgent,
        count, count, serror_rate, serror_rate,
        rerror_rate, rerror_rate, 1.0, 0.0, 0.0, 1, 1,
        1.0, 0.0, 1.0, 0.0, serror_rate, serror_rate,
        rerror_rate, rerror_rate
    ]

    feature_vector = scaler.transform([feature_vector])
    pred = model.predict(feature_vector)[0][0]
    label = 'attack' if pred > 0.5 else 'normal'
    features['prediction'] = label

    return features

In [17]:
def write_header_if_needed():
    try:
        with open(OUTPUT_FILE, "x", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=HEADERS)
            writer.writeheader()
    except FileExistsError:
        pass

In [18]:
def log_features_to_file(features):
    with open(OUTPUT_FILE, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writerow(features)

def flush_expired_flows():
    current_time = time.time()
    expired_keys = [k for k, t in flow_start_time.items() if current_time - t > FLOW_TIMEOUT]
    for key in expired_keys:
        if flows[key]:
            features = extract_flow_features(key, flows[key])
            log_features_to_file(features)
            print(f"Flow: {features['src']}->{features['dst']} | Prediction: {features['prediction']}")
        del flows[key]
        del flow_start_time[key]

In [19]:
def process_packet(pkt):
    if IP in pkt and (TCP in pkt or UDP in pkt):
        proto = 'tcp' if TCP in pkt else 'udp'
        ip = pkt[IP]
        l4 = pkt[TCP] if TCP in pkt else pkt[UDP]
        key = (ip.src, ip.dst, l4.sport, l4.dport, proto)

        flows[key].append(pkt)
        flow_start_time[key] = flow_start_time.get(key, pkt.time)

    flush_expired_flows()


In [20]:
# Main execution
write_header_if_needed()
print("Real-time Intrusion Detection Started... Press Ctrl+C to stop.")
try:
    sniff(prn=process_packet, store=False)
except KeyboardInterrupt:
    print("Stopping... Flushing remaining flows.")
    for key in list(flows.keys()):
        features = extract_flow_features(key, flows[key])
        log_features_to_file(features)
        print(f"Flow: {features['src']}->{features['dst']} | Prediction: {features['prediction']}")
    print("Finished. Log saved to", OUTPUT_FILE)


Real-time Intrusion Detection Started... Press Ctrl+C to stop.


c:\Users\Kimo Store\anaconda3\envs\OLD\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 74ms/step
Flow: 192.168.1.102->20.86.94.195 | Prediction: attack


c:\Users\Kimo Store\anaconda3\envs\OLD\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 16ms/step
Flow: 192.168.1.102->35.230.116.55 | Prediction: normal


...

1/1 [==============================] - 0s 15ms/step
Flow: 172.217.18.46->192.168.1.102 | Prediction: normal


c:\Users\Kimo Store\anaconda3\envs\OLD\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 17ms/step
Flow: 208.103.161.1->192.168.1.102 | Prediction: normal


c:\Users\Kimo Store\anaconda3\envs\OLD\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 22ms/step
Flow: 192.168.1.102->208.103.161.1 | Prediction: attack
